In [ ]:
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

## Importar bibliotecas

In [ ]:
from datetime import datetime, timedelta

import pandas as pd
import numpy as np
import requests as req
import warnings

In [ ]:
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

pd.set_option('mode.chained_assignment', None)

## Data schema

In [ ]:
categorical_selected_cols = ['origem', 'destino', 'metar', 'hora_ref', 'path', 'snapshot_radar']
numerical_selected_cols = ['troca', 'esperas']

selected_cols = categorical_selected_cols + numerical_selected_cols

## Param default

In [ ]:
paramsDefault = {
    'idate': '2022-06-02', # Data inicial da consulta | Example : AAAA-MM-DD
    'fdate': '2023-05-11' # Data final da consulta | Example : AAAA-MM-DD
}

## Funções auxiliares

In [ ]:
def adjust_timestamp(ts):
    timestamp_s = ts / 1000
    dt = datetime.utcfromtimestamp(timestamp_s)
    return dt.replace(minute=0, second=0, microsecond=0)

def getData(route, paramsWithoutToken = paramsDefault, token = 'a779d04f85c4bf6cfa586d30aaec57c44e9b7173'):
    """
    Fetches data from a specific API, handling exceptions and returning the data as a DataFrame.

    :param route: String representing the API endpoint.
    :param paramsWithoutToken: Dictionary containing the request parameters without the token. Defaults to paramsDefault.
    :param token: String containing the authentication token. Defaults to a fixed token.
    
    :return: A pandas DataFrame containing the data fetched from the API.
    
    :raises: May propagate exceptions related to the request if they occur.
    """
        
    url = f'http://montreal.icea.decea.mil.br:5002/api/v1/{route}'
    params = paramsWithoutToken
    params['token'] = token
    data = None
    
    while True:
        try:
            response = req.get(url, params)
            response.raise_for_status()
            data = response.json()
            break
            
        except:
            continue
    
    return pd.DataFrame(data)

## BINTRA

In [ ]:
def bimtra_ingestion():
    # Obtendo dados da fonte 'bimtra'
    df_bintra = getData('bimtra')

    # Removendo linhas em que a origem é igual ao destino, pois não formam uma rota válida
    df_bintra = df_bintra[df_bintra['origem'] != df_bintra['destino']]

    # Criando uma coluna 'rota' concatenando origem e destino para identificar a rota
    df_bintra['rota'] = df_bintra['origem'] + '_' + df_bintra['destino']

    # Criando colunas 'dt_dep_timestamp' e 'dt_arr_timestamp' para armazenar as datas de partida e chegada em formato de timestamp
    df_bintra['dt_dep_timestamp'] = df_bintra['dt_dep']
    df_bintra['dt_arr_timestamp'] = df_bintra['dt_arr']

    # Convertendo as colunas de timestamp em datetime
    df_bintra['dt_dep'] = pd.to_datetime(df_bintra['dt_dep_timestamp'], unit='ms')
    df_bintra['dt_arr'] = pd.to_datetime(df_bintra['dt_arr_timestamp'], unit='ms')

    # Calculando a duração da viagem em segundos e armazenando na coluna 'duration'
    df_bintra['duration'] = (df_bintra['dt_arr_timestamp'] - df_bintra['dt_dep_timestamp']) / 1000

    return df_bintra

# Obtendo dados da   fonte 'bimtra' e armazenando em df_bintra
df_bintra = bimtra_ingestion()

df_bintra.sample(3)

## CAT-62

In [ ]:

# Função para converter uma string de data e hora em timestamp Unix
def string_to_timestamp(datetime_str):
    """
    Converte uma string de data e hora no formato 'YYYY-MM-DD HH:mm:ss.SSS' em timestamp Unix.
    
    Parâmetros:
        datetime_str (str): String de data e hora no formato 'YYYY-MM-DD HH:mm:ss.SSS'.
        
    Retorna:
        int: Timestamp Unix.
    """
    dt_obj = datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S.%f')
    timestamp = int(dt_obj.timestamp())
    return timestamp

# Função para converter um timestamp Unix em uma string de data e hora
def timestamp_to_string(timestamp):
    """
    Converte um timestamp Unix em uma string de data e hora no formato 'YYYY-MM-DD HH:mm:ss.SSS'.
    
    Parâmetros:
        timestamp (int): Timestamp Unix.
        
    Retorna:
        str: String de data e hora no formato 'YYYY-MM-DD HH:mm:ss.SSS'.
    """
    dt_obj = datetime.fromtimestamp(timestamp)
    datetime_str = dt_obj.strftime('%Y-%m-%d %H:%M:%S.%f')
    return datetime_str

# Função para ingestão e processamento de dados da fonte 'cat-62'
def cat62_ingestion(start_timestamp, end_timestamp, step):
    """
    Obtém e processa os dados da fonte 'cat-62' dentro do intervalo de tempo especificado.

    Parâmetros:
        start_timestamp (int): Timestamp de início do intervalo.
        end_timestamp (int): Timestamp de término do intervalo.
        step (int): Passo de tempo em segundos.
        
    Retorna:
        DataFrame: DataFrame contendo as coordenadas do radar para cada voo.
    """
    df_cat_62_blocks = []
    for timestamp in range(start_timestamp, end_timestamp, step):
        dt_i = timestamp_to_string(timestamp)
        dt_f = timestamp_to_string(timestamp + step)
        print(dt_i)
        aux = getData('cat-62', {'idate': dt_i, 'fdate': dt_f})
        if len(aux) != 0:
            df_cat_62_blocks.append(aux)
    
    if df_cat_62_blocks:
        df_cat_62 = pd.concat(df_cat_62_blocks, ignore_index=True)
        return generate_snapshot_radar(df_cat_62)
    else:
        return pd.DataFrame(columns=['snapshot_radar', 'flightid'])

# Função para gerar as coordenadas do radar para cada voo
def generate_snapshot_radar(df):
    """
    Gera as coordenadas do radar para cada voo e armazena em um DataFrame.
    
    Parâmetros:
        df (DataFrame): DataFrame contendo os dados dos voos.
        
    Retorna:
        DataFrame: DataFrame com as coordenadas do radar para cada voo.
    """
    df_snapshot_radar = pd.DataFrame(columns=['snapshot_radar'])
    df_snapshot_radar.index.name = 'flightid'

    for flight in df['flightid'].unique():
        coords = "MULTIPOINT ("
        erase_comma = False
        for index, row in df[df['flightid'] == flight].iterrows():
            coords += f"({row['lon']} {row['lat']})"
            coords += ', '
            erase_comma = True
        if erase_comma:
            coords = coords[:-2]
        coords += ")"
        df_snapshot_radar.loc[flight, 'snapshot_radar'] = coords
        
    return df_snapshot_radar.reset_index()

# Definição dos timestamps de início e fim do intervalo e passo de tempo
start_timestamp = string_to_timestamp('2023-03-01 00:00:00.000')
end_timestamp = string_to_timestamp('2024-03-21 00:00:00.000')
step = 60 * 60 * 24

# Obtém os dados da fonte 'cat-62' e gera as coordenadas do radar para cada voo
df_snapshot_radar = cat62_ingestion(start_timestamp, end_timestamp, step)

# Exibe uma amostra dos dados do radar
df_snapshot_radar.sample(3)

In [ ]:
df_snapshot_radar.to_csv('../data/cat_62.csv', index=False)

## ESPERAS

In [ ]:
def esperas_ingestion():
    df_esperas = getData('esperas')
    df_esperas['hora_esperas'] = pd.to_datetime(df_esperas['hora'], unit='ms')
    df_esperas['destino'] = df_esperas['aero']
    df_esperas.drop(['hora', 'aero'], axis =1, inplace = True)
    
    return df_esperas

df_esperas = esperas_ingestion()
df_esperas.sample(3)

## METAF

In [ ]:
def metaf_ingestion():
    df_metaf = getData('metaf')
    df_metaf['hora_metaf'] = pd.to_datetime(df_metaf['hora'], unit='ms')
    df_metaf['aero_metaf'] = df_metaf['aero']
    df_metaf.drop(['hora', 'aero'], axis =1, inplace = True)

    return df_metaf

df_metaf = metaf_ingestion()
df_metaf.sample(3)

## METAR

In [ ]:
def metar_ingestion():
    df_metar = getData('metar')
    df_metar['hora_metar'] = pd.to_datetime(df_metar['hora'], unit='ms')
    df_metar['aero_metar'] = df_metar['aero']
    df_metar.drop(['hora', 'aero'], axis =1, inplace = True)

    return df_metar

metar_df = metar_ingestion()
metar_df.sample(3)

# SATELITE

In [ ]:
def satelite_ingestion():
    df_sat_met = getData('satelite')
    df_sat_met['hora_ref'] = pd.to_datetime(df_sat_met['data'])
    df_sat_met.drop(['data'], axis = 1, inplace = True)

    return df_sat_met

df_sat_met = satelite_ingestion()
df_sat_met.sample(3)

## TC-PREV

In [ ]:
def tcp_ingestion():
    df_tcp = getData('tc-prev')
    df_tcp['hora_tcp'] = pd.to_datetime(df_tcp['hora'], unit='ms')
    df_tcp['aero_tcp'] = 'SB' + df_tcp['aero']
    df_tcp.drop(['hora', 'aero'], axis =1, inplace = True)

    return df_tcp

df_tcp = tcp_ingestion()
df_tcp.sample(3)

## TC-REAL

In [ ]:
def tcr_ingestion():
    df_tcr = getData('tc-real')
    df_tcr['aero_tcr'] = 'SB' + df_tcr['aero']
    df_tcr['hora_tcr'] = df_tcr['hora'].apply(adjust_timestamp)
    aux = df_tcr.groupby(['hora_tcr', 'aero_tcr']).agg({'aero': ['count']}).reset_index()
    aux.columns = aux.columns.droplevel(1)
    aux['troca_real'] = aux['aero']
    df_tcr = aux.drop(['aero'], axis =1)

    return df_tcr

df_tcr = tcr_ingestion()
df_tcr.sample(3)

## JOIN

In [ ]:
def join_dataframes:
    df_bintra['hora_ref'] = df_bintra['dt_dep_timestamp'].apply(adjust_timestamp)
    df_bintra['hora_esperas'] = df_bintra['hora_ref'] - timedelta(hours=1)
    df_bintra['aero_esperas'] = df_bintra['destino']
    df_bintra['aero_metaf'] = df_bintra['destino']
    df_bintra['hora_metaf'] = df_bintra['hora_ref'] + timedelta(hours=1)
    df_bintra['aero_metar'] = df_bintra['destino']
    df_bintra['hora_metar'] = df_bintra['hora_ref']
    df_bintra['hora_tcp'] = df_bintra['hora_ref'] + timedelta(hours=1)
    df_bintra['aero_tcp'] = df_bintra['destino']
    df_bintra['hora_tcr'] = df_bintra['hora_ref']
    df_bintra['aero_tcr'] = df_bintra['destino']


    merged1_satelite = df_bintra.merge(df_sat_met, on='hora_ref', how='left')
    merged2_esperas = merged1_satelite.merge(df_esperas, on=['hora_esperas', 'destino'], how='left')
    merged3_metaf = merged2_esperas.merge(df_metaf, on=['hora_metaf', 'aero_metaf'], how='left')
    merged4_metar = merged3_metaf.merge(df_metar, on=['hora_metar', 'aero_metar'], how='left')
    merged5_tcp = merged4_metar.merge(df_tcp, on=['hora_tcp', 'aero_tcp'], how='left')
    merged6_tcr = merged5_tcp.merge(df_tcr, on=['hora_tcr', 'aero_tcr'], how='left').set_index('flightid')
    merged7_cat = merged6_tcr.merge(df_cat_62, on=['flightid'], how='left').set_index('flightid')
    data_train_origin = merged7_cat[
        ['origem', 'destino', 'metar', 'snapshot_radar','hora_ref', 'path', 'troca', 'esperas', 'duration']
    ]

    data_train_origin.loc[:, 'hora_ref'] = data_train_origin['hora_ref'].astype('str')

    data_train_origin.dropna(inplace=True)

    del df_bintra
    del merged1_satelite
    del merged2_esperas
    del merged3_metaf
    del merged4_metar
    del merged5_tcp
    del merged6_tcr
    del merged7_cat

    data_train_origin.to_csv('../data/data_train_origin.csv', index=True)


data_train = join_dataframes()
data_train.sample(3)


In [ ]:
data_train_origin.to_csv('../data/data_train_origin.csv', index=True)